# Creation de la base finale pour JS 

Votes et abstention

In [2]:
import pandas as pd
import geopandas as gpd
import os 
import matplotlib.pyplot as plt

In [4]:
dir=os.getcwd()

Densite de population

In [3]:
dens=pd.read_csv(dir+'\\brut\\metropole-densites-de-population-par-commune.csv',sep=';')
dens=dens.rename(columns={'Code INSEE':'insee',"Densité d'habitants (hab/km2)" : 'densitehabkm2'})
dens=dens[['insee','densitehabkm2']]

FileNotFoundError: [Errno 2] File /home/steve/Documents/ENSAE/Data_StoryTelling/dataviz\brut\metropole-densites-de-population-par-commune.csv does not exist: '/home/steve/Documents/ENSAE/Data_StoryTelling/dataviz\\brut\\metropole-densites-de-population-par-commune.csv'

In [ ]:
dens.insee=dens.insee.astype(int)
dens.dtypes

In [ ]:
A=dens.sort_values('insee').sort_values('insee')

In [ ]:
list(A.insee)

Niveau de vie et taux de pauvreté

In [ ]:
com=pd.read_csv(dir+'\\brut\\base-comparateur-de-territoires.csv',sep=';')
com=com.rename(columns={'CODGEO':'insee','MED16':'medianendv','TP6016':'tp'})
com=com[['insee','tp','medianendv']]

In [ ]:
com.insee=com.insee.astype(int)
com.dtypes

In [ ]:
com.head() # tp on a pas pour tout 

In [ ]:
len(com.insee.unique())

Pollution

In [ ]:
pol=pd.read_csv(dir+'\\brut\\indices_QA_commune_IDF_2017.csv',sep=',')

pol=pol.groupby('ninsee',as_index=False).mean()

pol=pol.rename(columns={'ninsee':'insee'})

In [ ]:
pol.insee=pol.insee.astype(int)

In [ ]:
pol.dtypes

Diplomes 

In [ ]:
dipl = pd.read_csv(dir+'\\brut\\base-cc-diplomes-formation-2016.csv',sep=';')

In [ ]:
dipl=dipl[['CODGEO','P16_NSCOL15P','P16_NSCOL15P_SUP']]

dipl=dipl.rename(columns={'P16_NSCOL15P':'nb_nscol','P16_NSCOL15P_SUP':'dipl_sup','CODGEO':'insee'})

dipl=dipl.assign(dipl_su_tx=dipl.dipl_sup/dipl.nb_nscol)

dipl=dipl[['insee','dipl_su_tx']]


In [ ]:
dipl.dtypes

In [ ]:
import numpy as np

In [ ]:
dipl=dipl[pd.to_numeric(dipl['insee'], errors='coerce').notnull()]

In [ ]:
dipl.insee=dipl.insee.astype(int)

Prix au mètre carré 

In [ ]:
prix=pd.read_csv(dir+'\\brut\\prixm2-communes-2017.csv',sep=',')

prix=prix[['INSEE_COM','Prixm2']]

prix=prix.rename(columns={'INSEE_COM':'insee','Prixm2':'prixm2'})

In [ ]:
prix.dtypes

In [ ]:
prix=prix[pd.to_numeric(prix['insee'], errors='coerce').notnull()]
prix.insee=prix.insee.astype(int)

In [ ]:
prix.dtypes

Impôt sur la fortune

In [ ]:
isf=pd.read_excel(dir+'\\brut\\ificom2018.xlsx',headers=True,sheet_name='définitif impot')

isf=isf[['Code commune (INSEE)','nombre de redevables','patrimoine moyen en €','impôt moyen en €']]

isf.columns=['insee','nb_redevables','patrimoineeuro','impotmoyeneuro']

In [ ]:
isf.dtypes

In [ ]:
isf['insee']= isf['insee'].transform(lambda x : x.replace(' ','')) 

In [ ]:
isf=isf[pd.to_numeric(isf['insee'], errors='coerce').notnull()]
isf.insee=isf.insee.astype(int)

In [ ]:
isf.dtypes

In [ ]:
#Base Sarah 
eco=pd.read_csv(dir+'\\brut\\data_chom_prop.csv')
eco.head()

In [ ]:
eco=eco.drop(columns=['Unnamed: 0'])
eco=eco.rename(columns={'CODGEO':'insee'})

In [ ]:
eco.dtypes

Votes

In [ ]:
#vote =pd.read_csv(dir+'\\brut\\data_final_vote.csv')
#vote=vote[['CODGEO','Tour1Candidat1']]

In [ ]:
#vote=vote.rename(columns={'CODGEO':'insee'})

In [ ]:
# Vote à la mano 
vote = pd.read_excel(dir+'\\brut\\vote_mano.xlsx')

Merge 

In [ ]:
bases=[eco,dens,dipl,pol,vote,com,isf,prix]

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')
variables= pd.DataFrame(pop.insee)
variables.insee=variables.insee.astype(int)

In [ ]:
pop.plot()

In [ ]:
for data in bases:
    variables=variables.merge(data,how='left',on='insee')

In [ ]:
variables.head()

In [ ]:
# SOucis avec les élections j'ai pas les correspondances !! 

In [ ]:
variables.drop_duplicates( inplace = True) 

In [ ]:
#variables.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
variables = variables.round(2)

In [ ]:
variables.to_csv('variables.csv')

Creation du CSV our la population slider

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')

In [ ]:
pop=pop[['insee','year','population','pct_ch','steps']]

In [ ]:
pop.to_csv('population_csv.csv')

In [ ]:
pop[pop.insee=='78561']

Professionnels de Santé

In [173]:
med=pd.read_csv(dir+'\\brut\\annuaire-et-localisation-des-professionnels-de-sante.csv',sep=';')

med=med[["Nature de l'exercice","Convention du professionnel","Coordonnées",'Adresse','Code INSEE']]

med.columns=['nature_exercice','convention','coordonnees','adresse','insee']

In [174]:
# Ici interessant de mettre des marqueurs ? Je laisse comme ça => sinon compter 
count_med=med.groupby('insee',as_index=False)['coordonnees'].count()

In [175]:
med.dtypes

nature_exercice    object
convention         object
coordonnees        object
adresse            object
insee               int64
dtype: object

In [176]:
count_med.dtypes

insee          int64
coordonnees    int64
dtype: object

In [177]:
med.insee=med.insee.astype(int)

In [178]:
med.columns

Index(['nature_exercice', 'convention', 'coordonnees', 'adresse', 'insee'], dtype='object')

In [179]:
med=med[['nature_exercice','coordonnees','insee']]

In [180]:
med=med.merge(count_med,on='insee')

In [181]:
med.columns=['nature','coordonnees','insee','nombre']

In [182]:
new= med["coordonnees"].str.split(",", n = 1, expand = True) 

In [183]:
med_data['lat'] = new[0]
med_data['lon'] = new[1]

In [184]:
med_data 

,nature,coordonnees,insee,nombre,geometry,new,lat,lon
0,Libéral activité salariée,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.970077,48.829281,2.3568
1,Libéral intégral,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.970077,48.824815,2.357629
2,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.96271,48.883061,2.329228
3,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.96271,48.833789,2.308108
4,Libéral activité salariée,48.971396,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ...",48.971396,48.853281,2.293727
...,...,...,...,...,...,...,...,...
7773,Libéral intégral,49.014456,77241,4,"POLYGON ((2.67922 49.01336, 2.68390 49.01933, ...",49.014456,48.857847,2.304228
7774,Libéral intégral,48.704353,91136,1,"POLYGON ((2.25880 48.70794, 2.25881 48.70836, ...",48.704353,48.854348,2.383557
7775,Libéral intégral,49.064301,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ...",49.064301,48.844213,2.342242
7776,Libéral intégral,49.064414,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ...",49.064414,48.865643,2.288865


In [153]:
def df_to_geojson(df, properties, lat='lat', lon='lon'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [154]:
cols = ['insee','nombre','nature']
geojson = df_to_geojson(med_data, cols)

In [160]:
import json

with open('med_data.geojson', 'a') as f:
    json.dump(geojson, f)

In [77]:
#med_data.to_csv('med_data.csv')

In [86]:
med_data["coordonnees"]= med_data["coordonnees"].str.split(",", n = 2, expand = True) 

In [85]:
med_data

,nature,coordonnees,insee,nombre,geometry
0,Libéral activité salariée,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
1,Libéral intégral,48.970077,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
2,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
3,Libéral intégral,48.96271,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
4,Libéral activité salariée,48.971396,95197,32,"POLYGON ((2.30969 48.97072, 2.30979 48.97119, ..."
...,...,...,...,...,...
7773,Libéral intégral,49.014456,77241,4,"POLYGON ((2.67922 49.01336, 2.68390 49.01933, ..."
7774,Libéral intégral,48.704353,91136,1,"POLYGON ((2.25880 48.70794, 2.25881 48.70836, ..."
7775,Libéral intégral,49.064301,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ..."
7776,Libéral intégral,49.064414,77322,2,"POLYGON ((2.57098 49.06058, 2.57141 49.06093, ..."


In [185]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')

In [186]:
pop=pop[['insee','geometry']]

In [187]:
pop=pop.drop_duplicates()

In [188]:
pop.insee=pop.insee.astype(int)

In [192]:
med_merge=med_data[['nature','nombre','insee']]

In [193]:
med_merge = med_merge.merge(pop)

In [202]:
med_merge.columns

Index(['nature', 'nombre', 'insee', 'geometry'], dtype='object')

In [203]:
from geopandas import GeoDataFrame
med_merge = GeoDataFrame(med_merge, geometry=med_merge['geometry'])

In [207]:
med_merge.to_file("med_merge.geojson", driver='GeoJSON')

In [206]:
med_merge=med_merge[['insee','nombre','geometry']]

In [213]:
med_merge.nombre.max()

393

# Traffic (lié aux stations et non aux communes)

In [219]:
traffic_par_heure=pd.read_csv('traffic_par_heure.csv')

In [247]:
import geopandas as gpd 
stations=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\stations.geojson')

In [248]:
stations=stations.merge(traffic_par_heure,left_on='nom_long',right_on='LIBELLE_ARRET',how='right')

In [250]:
stations=stations[['LIBELLE_ARRET','TRNC_HORR_60','NB_VALD_HOUR','pourc_validations','geometry']]

In [251]:
stations=stations.dropna()

In [252]:
stations=stations.drop_duplicates()

In [253]:
stations

,LIBELLE_ARRET,TRNC_HORR_60,NB_VALD_HOUR,pourc_validations,geometry
0,MASSY-PALAISEAU,17H-18H,193416.439728,11.01,POINT (2.25828 48.72469)
9,MASSY-PALAISEAU,3H-4H,175.673424,0.01,POINT (2.25828 48.72469)
18,MASSY-PALAISEAU,12H-13H,62715.412337,3.57,POINT (2.25828 48.72469)
27,MASSY-PALAISEAU,16H-17H,112958.011576,6.43,POINT (2.25828 48.72469)
36,MASSY-PALAISEAU,1H-2H,351.346848,0.02,POINT (2.25828 48.72469)
...,...,...,...,...,...
7621,SEVRAN-LIVRY,13H-14H,14204.027049,3.45,POINT (2.53471 48.93628)
7626,SEVRAN-LIVRY,1H-2H,288.197650,0.07,POINT (2.53471 48.93628)
7631,SEVRAN-LIVRY,4H-5H,25155.537760,6.11,POINT (2.53471 48.93628)
7636,SEVRAN-LIVRY,6H-7H,59039.347213,14.34,POINT (2.53471 48.93628)


In [254]:
new=stations.TRNC_HORR_60.str.split("H", n = 1, expand = True) 

In [255]:
stations['heure']=new[0]

In [256]:
stations=stations[['LIBELLE_ARRET','NB_VALD_HOUR','heure','geometry']]

In [258]:
from geopandas import GeoDataFrame
stations2 = GeoDataFrame(stations, geometry=med_merge['geometry'])

In [259]:
stations2.to_file("stations2.geojson", driver='GeoJSON')

Cartes pour visualiser le résultat 

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')
pop.insee=pop.insee.astype(int)
data_map=variables.merge(pop,on='insee')

In [ ]:
data_map.columns

In [ ]:
import mapclassify
import geoplot
import matplotlib.pyplot as plt 
data_map=data_map.set_geometry('geometry')

In [ ]:
scheme = mapclassify.Quantiles(data_map.TauxChomage, k=10)
geoplot.choropleth(
    data_map, hue=data_map.TauxChomage, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux de chômage')

In [ ]:
scheme = mapclassify.Quantiles(data_map.TauxProprietaires, k=10)
geoplot.choropleth(
    data_map, hue=data_map.TauxProprietaires, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux de propriétaires')

In [ ]:
scheme = mapclassify.Quantiles(data_map.pm10, k=10)
geoplot.choropleth(
    data_map, hue=data_map.pm10, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux pollution pm10')

In [ ]:
scheme = mapclassify.Quantiles(data_map.dipl_su_tx, k=10)
geoplot.choropleth(
    data_map, hue=data_map.dipl_su_tx, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('dipl_su_tx')

In [ ]:

geoplot.choropleth(
    data_map, hue=data_map.Tour1Candidat1,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('vote_politique')